In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.types import StructType
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf, monotonically_increasing_id

import os 

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /usr/share/java/mariadb-java-client-2.5.3.jar pyspark-shell'

Configuramos la sesión de pyspark

In [3]:
spark_context = SparkContext()
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

### Cargamos las bases desde los csv

En la tabla aeropuertos eliminamos los registros con fechas de construccióin superior a 2013 y eliminamos los registros repetidos.

In [4]:
from pyspark.sql.types import IntegerType,DateType,DoubleType, StringType

df_aeropuertos = spark.read.load("aeropuertosEtapa3.csv",format="csv", sep=",", inferSchema="true", header="true")
df_aeropuertos = df_aeropuertos.replace("nan", None)
df_aeropuertos = df_aeropuertos.drop("_c0")
df_aeropuertos = df_aeropuertos.withColumn("pbmo",df_aeropuertos.pbmo.cast(DoubleType()))
df_aeropuertos = df_aeropuertos.withColumn("fecha_construccion",df_aeropuertos.fecha_construccion.cast(DateType()))
df_aeropuertos = df_aeropuertos.withColumn("fecha_vigencia",df_aeropuertos.fecha_vigencia.cast(DateType()))
df_aeropuertos = df_aeropuertos.withColumn("numero_vuelos_origen",df_aeropuertos.numero_vuelos_origen.cast(IntegerType()))
df_aeropuertos = df_aeropuertos.withColumn("gcd_departamento",df_aeropuertos.gcd_departamento.cast(StringType()))
df_aeropuertos = df_aeropuertos.withColumn("gcd_municipio",df_aeropuertos.gcd_municipio.cast(StringType()))
df_aeropuertos = df_aeropuertos.drop_duplicates(subset=['sigla', "Ano"])
df_aeropuertos.show()

+-----+----+--------------------+--------------------+------------+-------------+-------+--------+--------------------+--------------------+--------------+-----------+------+---------+----------+------------------+--------------+-----+----------+--------------------+----------------+-------------+----+
|sigla|iata|              nombre|           municipio|departamento|    categoria|latitud|longitud|         propietario|          explotador|longitud_pista|ancho_pista|  pbmo|elevacion|resolucion|fecha_construccion|fecha_vigencia|clase|      tipo|numero_vuelos_origen|gcd_departamento|gcd_municipio| Ano|
+-----+----+--------------------+--------------------+------------+-------------+-------+--------+--------------------+--------------------+--------------+-----------+------+---------+----------+------------------+--------------+-----+----------+--------------------+----------------+-------------+----+
|  9AO|null|         SANTA CLARA|         Sabanalarga|    Casanare|    Aeródromo| 4.4172

In [5]:
df_vuelos = spark.read.load("vuelosEtapa3.csv",format="csv", sep=",", inferSchema="true", header="true")
df_vuelos = df_vuelos.replace("nan", None)
df_vuelos = df_vuelos.withColumn("sillas",df_vuelos.sillas.cast(IntegerType()))
df_vuelos = df_vuelos.withColumn("pasajeros",df_vuelos.pasajeros.cast(IntegerType()))
df_vuelos = df_vuelos.withColumn("vuelos",df_vuelos.vuelos.cast(IntegerType()))
df_vuelos.show()

+----+---+------+-------+-----------+----------+-------+--------------------+------+------+--------------+---------+-----------+
| ano|mes|origen|destino|tipo_equipo|tipo_vuelo|trafico|             empresa|vuelos|sillas|carga_ofrecida|pasajeros|carga_bordo|
+----+---+------+-------+-----------+----------+-------+--------------------+------+------+--------------+---------+-----------+
|2016| 11|   BOG|    MDE|       R722|         R|      N|                 LAS|     3|     0|       72000.0|        0|    60146.0|
|2014|  2|   EOH|    BGA|       JS32|         R|      N|AEROLINEA DE ANTI...|    45|   766|      112220.0|      439|      999.0|
|2017|  4|   VDF|    EOH|       C210|         T|      N|        HELI JET SAS|    15|     0|           0.0|       60|        0.0|
|2018| 12|   PEI|    EOH|       C303|         T|      N|        HELI JET SAS|     4|     0|           0.0|        3|        0.0|
|2018| 12|   LMC|    BOG|       JS32|         T|      N|         SARPA S.A.S|     2|     0|      

In [6]:
df_poblacion = spark.read.csv("demografico.txt", sep = "|", header = True, encoding = "ISO-8859-1")
df_divipol = spark.read.load("divipola.csv",format="csv", sep=";", inferSchema="true", header="true")

df_poblacion = df_poblacion.withColumnRenamed("Código Departamento", "codigo_departamento")
df_poblacion = df_poblacion.withColumnRenamed("Código Entidad", "codigo_municipio")

df_divipol = df_divipol.withColumnRenamed("Código Departamento", "codigo_departamento")
df_divipol = df_divipol.withColumnRenamed("Código Municipio", "codigo_municipio")

df_poblacion = df_poblacion.join(df_divipol, how = "inner", on =["codigo_departamento", "codigo_municipio"])
df_poblacion = df_poblacion.withColumnRenamed("Código Centro Poblado", "codigo_centro_poblado")

df_poblacion = df_poblacion.filter((df_poblacion.Indicador == 'Población total de hombres') | (df_poblacion.Indicador == 'Población total de mujeres'))
df_poblacion = df_poblacion.filter(df_poblacion.Año < 2021)

df_poblacion.drop_duplicates(["codigo_centro_poblado", "Indicador", "Año"])
df_poblacion.show(5)


+-------------------+----------------+------------+---------+--------------------+--------------------+--------------------+-------------+----------------+----+---+--------------------+----------------+---------------------+-------------------+----------------+---------------------+----+
|codigo_departamento|codigo_municipio|Departamento|  Entidad|           Dimensión|        Subcategoría|           Indicador|Dato Numérico|Dato Cualitativo| Año|Mes|              Fuente|Unidad de Medida|codigo_centro_poblado|Nombre Departamento|Nombre Municipio|Nombre Centro Poblado|Tipo|
+-------------------+----------------+------------+---------+--------------------+--------------------+--------------------+-------------+----------------+----+---+--------------------+----------------+---------------------+-------------------+----------------+---------------------+----+
|                 15|           15001|      Boyacá|    Tunja|Demografía y pobl...|Población desagre...|Población total d...|    81.43

In [7]:
df_poblacion = spark.read.csv("demografico.txt", sep = "|", header = True, encoding = "ISO-8859-1")
df_poblacion = df_poblacion.filter((df_poblacion.Indicador == 'Población total de hombres') | (df_poblacion.Indicador == 'Población total de mujeres'))
df_poblacion = df_poblacion.filter(df_poblacion.Año < 2021)

df_divipol = spark.read.load("divipola.csv",format="csv", sep=";", inferSchema="true", header="true")
df_divipol = df_divipol.filter(df_divipol.Tipo == "CM")
df_poblacion = df_poblacion.withColumnRenamed("Código Departamento", "codigo_departamento")
df_poblacion = df_poblacion.withColumnRenamed("Código Entidad", "codigo_municipio")
df_divipol = df_divipol.withColumnRenamed("Código Departamento", "codigo_departamento")
df_divipol = df_divipol.withColumnRenamed("Código Municipio", "codigo_municipio")

df_poblacion = df_poblacion.withColumn("codigo_departamento",df_poblacion.codigo_departamento.cast(IntegerType()))
df_poblacion = df_poblacion.withColumn("codigo_municipio",df_poblacion.codigo_municipio.cast(IntegerType()))

df_divipol = df_divipol.withColumn("codigo_departamento",df_divipol.codigo_departamento.cast(IntegerType()))
df_divipol = df_divipol.withColumn("codigo_municipio",df_divipol.codigo_municipio.cast(IntegerType()))

df_poblacion = df_poblacion.join(df_divipol, how = "left", on =["codigo_departamento", "codigo_municipio"])
df_poblacion = df_poblacion.withColumnRenamed("Código Centro Poblado", "codigo_centro_poblado")

df_poblacion.show(5)
df_poblacion.count()

+-------------------+----------------+------------+---------+--------------------+--------------------+--------------------+-------------+----------------+----+---+--------------------+----------------+---------------------+-------------------+----------------+---------------------+----+
|codigo_departamento|codigo_municipio|Departamento|  Entidad|           Dimensión|        Subcategoría|           Indicador|Dato Numérico|Dato Cualitativo| Año|Mes|              Fuente|Unidad de Medida|codigo_centro_poblado|Nombre Departamento|Nombre Municipio|Nombre Centro Poblado|Tipo|
+-------------------+----------------+------------+---------+--------------------+--------------------+--------------------+-------------+----------------+----+---+--------------------+----------------+---------------------+-------------------+----------------+---------------------+----+
|                 15|           15000|      Boyacá|   Boyacá|Demografía y pobl...|Población desagre...|Población total d...|   599.29

6804

In [8]:
cond = [(df_poblacion.codigo_departamento == df_divipol.codigo_departamento) & (df_poblacion.codigo_municipio == df_divipol.codigo_municipio)]
df_poblacion.join(df_divipol, cond, 'inner').count()

6606

In [9]:
df_divipol = spark.read.load("divipola.csv",format="csv", sep=";", inferSchema="true", header="true")
df_divipol.show()

+-------------------+----------------+---------------------+-------------------+----------------+---------------------+----+
|Código Departamento|Código Municipio|Código Centro Poblado|Nombre Departamento|Nombre Municipio|Nombre Centro Poblado|Tipo|
+-------------------+----------------+---------------------+-------------------+----------------+---------------------+----+
|                  5|            5001|              5001000|          ANTIOQUIA|        MEDELLÍN|             MEDELLÍN|  CM|
|                  5|            5001|              5001001|          ANTIOQUIA|        MEDELLÍN|             PALMITAS|   C|
|                  5|            5001|              5001004|          ANTIOQUIA|        MEDELLÍN|          SANTA ELENA|   C|
|                  5|            5001|              5001005|          ANTIOQUIA|        MEDELLÍN|        PEDREGAL ALTO| IPM|
|                  5|            5001|              5001009|          ANTIOQUIA|        MEDELLÍN|            ALTAVISTA|   C|


In [10]:
df_cobertura = spark.read.load("Cobertura.csv",format="csv", sep=";", inferSchema="true", header="true")
df_cobertura = df_cobertura.withColumnRenamed("Codigo Centro Poblado", "codigo_centro_poblado")
df_cobertura = df_cobertura.withColumnRenamed("Distancia(Km)", "distancia")

df_cobertura.show()



+---------------------+---------------------+----------+-----------+---------+---------+-----------+--------+-----------+--------+-----------+-------------+---------------+--------------+
|codigo_centro_poblado|Nombre centro poblado|Aeropuerto|  distancia|Cobertura|Aerodromo|D_Aerodromo|Regional| D_Regional|Nacional| D_Nacional|Internacional|D_Internacional|Tipo_Cobertura|
+---------------------+---------------------+----------+-----------+---------+---------+-----------+--------+-----------+--------+-----------+-------------+---------------+--------------+
|              5001000|             MEDELLÍN|       EOH| 3,07967291|     true|      7IC|52,67179641|     URR|62,45094172|     EOH| 3,07967291|          MDE|    19,78362876| Internacional|
|              5001001|             PALMITAS|       EOH|17,55454828|     true|      7FX|61,18342718|     URR|49,85921309|     EOH|17,55454828|          MDE|    35,59064494| Internacional|
|              5001004|          SANTA ELENA|       EOH|9,85

### Creamos tanto las dimensiones como las tablas de hechos

In [11]:
def dimension_fecha(aeropuertos, vuelos):
    fecha = vuelos.select("ano", "mes").distinct().dropna()
    fecha = fecha.withColumn("id_fecha", monotonically_increasing_id()+1)
    def trimestre (mes):
        if mes < 4:
            return(1)
        elif mes < 7:
            return(2)
        elif mes < 10:
            return(3)
        else:
            return(4)
    trimestre_udf = udf(lambda x: trimestre(x), IntegerType())
    fecha = fecha.withColumn("trimestre", trimestre_udf(fecha["mes"]))
    
    return(fecha)

def dimension_aerolinea(aeropuertos, vuelos):
    aerolinea = vuelos.select("empresa").distinct().sort("empresa").dropna()
    aerolinea = aerolinea.withColumn("id_aerolinea", monotonically_increasing_id()+1)
    return(aerolinea)

def dimension_operacion(aeropuertos, vuelos):
    operacion = vuelos.select("tipo_vuelo").distinct().dropna()
    operacion = operacion.withColumn("id_operacion", monotonically_increasing_id()+1)
    return(operacion)

def dimension_avion(aeropuertos, vuelos):
    avion = vuelos.select("tipo_equipo").distinct().dropna()
    avion = avion.withColumn("id_avion", monotonically_increasing_id()+1)
    return(avion)

def dimension_trafico(aeropuertos, vuelos):
    trafico = vuelos.select("trafico").distinct().dropna()
    trafico = trafico.withColumn("id_trafico", monotonically_increasing_id()+1)
    return(trafico)

def dimension_aeropuerto_historia(aeropuertos, vuelos):
    aeropuertos_df = aeropuertos.replace("nan", None)
    aeropuertos_df = aeropuertos_df.drop("_c0")
    aeropuertos_df = aeropuertos_df.withColumn("pbmo",aeropuertos_df.pbmo.cast(DoubleType()))
    aeropuertos_df = aeropuertos_df.withColumn("fecha_construccion",aeropuertos_df.fecha_construccion.cast(DateType()))
    aeropuertos_df = aeropuertos_df.withColumn("fecha_vigencia",aeropuertos_df.fecha_vigencia.cast(DateType()))
    aeropuertos_df = aeropuertos_df.withColumn("numero_vuelos_origen",aeropuertos_df.numero_vuelos_origen.cast(IntegerType()))
    aeropuertos_df = aeropuertos_df.withColumn("Ano",aeropuertos_df.Ano.cast(IntegerType()))
    aeropuertos_df = aeropuertos_df.withColumn("gcd_departamento",aeropuertos_df.gcd_departamento.cast(StringType()))
    aeropuertos_df = aeropuertos_df.withColumn("gcd_municipio",aeropuertos_df.gcd_municipio.cast(StringType()))

    Inicio_udf = udf(lambda x: str(x) +'-01-01', StringType())
    Final_udf = udf(lambda x: str(x) +'-12-31', StringType())
    aeropuertos_df = aeropuertos_df.withColumn("InicioVigencia", Inicio_udf(aeropuertos_df["Ano"]))
    aeropuertos_df = aeropuertos_df.withColumn("InicioVigencia",aeropuertos_df.InicioVigencia.cast(DateType()))
    aeropuertos_df = aeropuertos_df.withColumn("FinalVigencia", Final_udf(aeropuertos_df["Ano"]))
    aeropuertos_df = aeropuertos_df.withColumn("FinalVigencia",aeropuertos_df.FinalVigencia.cast(DateType()))

    aeropuertos_df = aeropuertos_df.withColumn('Actual', functions.when(aeropuertos_df['Ano'] == aeropuertos_df.select("Ano").rdd.max()[0], "S").otherwise("N"))

    aeropuertos_df = aeropuertos_df.withColumn("id_aeropuerto", monotonically_increasing_id()+1)
    aeropuertos_df = aeropuertos_df.withColumnRenamed('Ano','ano')

    return(aeropuertos_df)

def dimension_lugar(poblacion, cobertura):
    lugar_df = poblacion.select("codigo_centro_poblado","Departamento","codigo_municipio","codigo_departamento",
                               "Nombre Departamento", "Nombre Municipio")
    lugar_df = lugar_df.join(cobertura, how = "inner", on ="codigo_centro_poblado")
    lugar_df = lugar_df.select("codigo_centro_poblado","codigo_municipio","codigo_departamento",
                               "Nombre Departamento", "Nombre Municipio", "Tipo_Cobertura")
    lugar_df = lugar_df.withColumnRenamed("Tipo_Cobertura", "tipo_Cobertura")
    lugar_df = lugar_df.withColumnRenamed("Nombre Departamento", "Departamento")
    lugar_df = lugar_df.withColumnRenamed("Nombre Municipio", "Municipio")
    lugar_df = lugar_df.distinct()
    lugar_df = lugar_df.withColumn("id_centro_poblado", monotonically_increasing_id()+1)
    
    return(lugar_df)
    
def hechos_cobertura(poblacion, cobertura, aeropuerto, lugar):
    cobertura_df = poblacion.withColumn('Dato Numérico', functions.regexp_replace(poblacion["Dato Numérico"], r"\.", ""))
    cobertura_df = cobertura_df.withColumn('Dato Numérico', functions.regexp_replace(cobertura_df["Dato Numérico"], ",", "."))
    cobertura_df = cobertura_df.withColumn("Dato Numérico",cobertura_df["Dato Numérico"].cast(IntegerType()))
    
    cobertura_df = cobertura_df.join(cobertura, how = "inner", on ="codigo_centro_poblado")
    
    cobertura_df = cobertura_df.groupBy("Año", "distancia", "codigo_centro_poblado", "Aeropuerto").pivot("Indicador").sum("Dato Numérico")
    poblacion_udf = udf(lambda x, y: x+y, IntegerType())
    cobertura_df = cobertura_df.withColumn("poblacion", poblacion_udf(cobertura_df["Población total de hombres"], cobertura_df["Población total de mujeres"]))
    cobertura_df = cobertura_df.withColumnRenamed("Población total de hombres", "poblacion_hombres")
    cobertura_df = cobertura_df.withColumnRenamed("Población total de mujeres", "poblacion_mujeres")
    cobertura_df = cobertura_df.withColumnRenamed("Aeropuerto", "sigla")
    
    cobertura_df = cobertura_df.join(aeropuerto.filter(aeropuerto.Actual == "S"), how='inner', on='sigla')
    cobertura_df = cobertura_df.select("Año", "distancia", "codigo_centro_poblado", "id_aeropuerto",
                                      "poblacion_hombres", "poblacion_mujeres", "poblacion")
    
    cobertura_df = cobertura_df.join(lugar, how='left', on='codigo_centro_poblado')
    cobertura_df = cobertura_df.select("Año", "distancia", "id_centro_poblado", "id_aeropuerto",
                                      "poblacion_hombres", "poblacion_mujeres", "poblacion")
    
    cobertura_df = cobertura_df.drop_duplicates(subset=['Año', "id_centro_poblado"])
    
    cobertura_df = cobertura_df.withColumn('id_aeropuerto', functions.when(cobertura_df['id_aeropuerto'] == 'null', '-1').otherwise(cobertura_df['id_aeropuerto']))
    cobertura_df = cobertura_df.withColumn('id_centro_poblado', functions.when(cobertura_df['id_centro_poblado'] == 'null', '-1').otherwise(cobertura_df['id_centro_poblado']))

    return(cobertura_df)
    

def tabla_hechos_vuelos(vuelos, aerolinea,operacion, avion, trafico, fecha, aeropuerto):
    vuelos = vuelos.join(aerolinea, how='inner', on='empresa')
    vuelos = vuelos.join(operacion, how='inner', on='tipo_vuelo')
    vuelos = vuelos.join(avion, how='inner', on='tipo_equipo')
    vuelos = vuelos.join(trafico, how='inner', on='trafico')
    vuelos = vuelos.join(fecha, how='inner', on=["ano", "mes"])
    vuelos = vuelos.join(aeropuerto.withColumnRenamed('sigla', 'origen'), how='inner', on = ["origen", "ano"])
    vuelos = vuelos.withColumnRenamed("id_aeropuerto", "id_origen")
    vuelos = vuelos.join(aeropuerto.withColumnRenamed('sigla', 'destino'), how='inner', on = ["destino", "ano"])
    vuelos = vuelos.withColumnRenamed("id_aeropuerto", "id_destino")

    
    tabla_hechos = vuelos.select("vuelos", "sillas", "pasajeros", "carga_ofrecida", "carga_bordo",
                                "id_aerolinea", "id_operacion", "id_avion", "id_trafico", "id_fecha", "id_origen", "id_destino")
    
    tabla_hechos = tabla_hechos.withColumn('id_aerolinea', functions.when(tabla_hechos['id_aerolinea'] == 'null', '-1').otherwise(tabla_hechos['id_aerolinea']))
    tabla_hechos = tabla_hechos.withColumn('id_operacion', functions.when(tabla_hechos['id_operacion'] == 'null', '-1').otherwise(tabla_hechos['id_operacion']))
    tabla_hechos = tabla_hechos.withColumn('id_avion', functions.when(tabla_hechos['id_avion'] == 'null', '-1').otherwise(tabla_hechos['id_avion']))
    tabla_hechos = tabla_hechos.withColumn('id_trafico', functions.when(tabla_hechos['id_trafico'] == 'null', '-1').otherwise(tabla_hechos['id_trafico']))
    tabla_hechos = tabla_hechos.withColumn('id_fecha', functions.when(tabla_hechos['id_fecha'] == 'null', '-1').otherwise(tabla_hechos['id_fecha']))
    tabla_hechos = tabla_hechos.withColumn('id_origen', functions.when(tabla_hechos['id_origen'] == 'null', '-1').otherwise(tabla_hechos['id_origen']))
    tabla_hechos = tabla_hechos.withColumn('id_destino', functions.when(tabla_hechos['id_destino'] == 'null', '-1').otherwise(tabla_hechos['id_destino']))
    
    return(tabla_hechos)

### Creamos el proceso de ETL

en este paso llamamos las funciones creadas anteriormente y guardamos cada tabla en un csv diferente. 

In [12]:
def ETL(aeropuertos, vuelos, poblacion, cobertura):
    PATH='modelo/'
    
    fecha = dimension_fecha(aeropuertos, vuelos)
    fecha.toPandas().to_csv(PATH+'dimension_fecha.csv')
    print("Dimensión Fecha: \n")
    print("Las dimensiones de la base de datos son: ", (fecha.count(), len(fecha.columns)), "\n")
    print("Las columnas son: ", fecha.columns, "\n")
    print(fecha.show(5))
    print("\n")
    
    
    aerolinea = dimension_aerolinea(aeropuertos, vuelos)
    aerolinea.toPandas().to_csv(PATH+'dimension_aerolinea.csv')
    print("Dimensión aerolinea: \n")
    print("Las dimensiones de la base de datos son: ", (aerolinea.count(), len(aerolinea.columns)), "\n")
    print("Las columnas son: ", aerolinea.columns, "\n")
    print(aerolinea.show(5))
    print("\n")
    
    operacion = dimension_operacion(aeropuertos, vuelos)
    operacion.toPandas().to_csv(PATH+'dimension_operacion.csv')
    print("Dimensión operacion: \n")
    print("Las dimensiones de la base de datos son: ", (operacion.count(), len(operacion.columns)), "\n")
    print("Las columnas son: ", operacion.columns, "\n")
    print(operacion.show(5))
    print("\n")
    
    avion = dimension_avion(aeropuertos, vuelos)
    avion.toPandas().to_csv(PATH+'dimension_avion.csv')
    print("Dimensión avion: \n")
    print("Las dimensiones de la base de datos son: ", (avion.count(), len(avion.columns)), "\n")
    print("Las columnas son: ", avion.columns, "\n")
    print(avion.show(5))
    print("\n")
    
    trafico = dimension_trafico(aeropuertos, vuelos)
    trafico.toPandas().to_csv(PATH+'dimension_trafico.csv')
    print("Dimensión trafico: \n")
    print("Las dimensiones de la base de datos son: ", (trafico.count(), len(trafico.columns)), "\n")
    print("Las columnas son: ", trafico.columns, "\n")
    print(trafico.show(5))
    print("\n")
    
    aeropuerto_historia = dimension_aeropuerto_historia(aeropuertos, vuelos)
    aeropuerto_historia.toPandas().to_csv(PATH+'dimension_aeropuerto_historia.csv')
    print("Dimensión aeropuerto con historia: \n")
    print("Las dimensiones de la base de datos son: ", (aeropuerto_historia.count(), len(aeropuerto_historia.columns)), "\n")
    print("Las columnas son: ", aeropuerto_historia.columns, "\n")
    print(aeropuerto_historia.show(5))
    print("\n")
    
      
    lugar = dimension_lugar(poblacion, cobertura)
    lugar.toPandas().to_csv(PATH+'dimension_lugar.csv')
    print("Dimensión lugar: \n")
    print("Las dimensiones de la base de datos son: ", (lugar.count(), len(lugar.columns)), "\n")
    print("Las columnas son: ", lugar.columns, "\n")
    print(lugar.show(5))
    print("\n")
    
    tabla_hechos_cobertura = hechos_cobertura(poblacion, cobertura, aeropuerto_historia, lugar)
    tabla_hechos_cobertura.toPandas().to_csv(PATH+'tabla_hechos_cobertura.csv')
    print("Tabla de Hechos de Cobertura: \n")
    print("Las dimensiones de la base de datos son: ", (tabla_hechos_cobertura.count(), len(tabla_hechos_cobertura.columns)), "\n")
    print("Las columnas son: ", tabla_hechos_cobertura.columns, "\n")
    print(tabla_hechos_cobertura.show(5))
    print(tabla_hechos_cobertura.describe().show())
    print("\n")
    
    tabla_hechos = tabla_hechos_vuelos(vuelos, aerolinea,operacion, avion, trafico, fecha, aeropuerto_historia)
    tabla_hechos.toPandas().to_csv(PATH+'tabla_hechos.csv')
    print("Tabla de Hechos de Vuelos: \n")
    print("Las dimensiones de la base de datos son: ", (tabla_hechos.count(), len(tabla_hechos.columns)), "\n")
    print("Las columnas son: ", tabla_hechos.columns, "\n")
    print(tabla_hechos.show(5))
    print(tabla_hechos.describe().show())
    print("\n")

## Ejecutamos lo construido

In [13]:
ETL(df_aeropuertos, df_vuelos, df_poblacion, df_cobertura)


Dimensión Fecha: 

Las dimensiones de la base de datos son:  (108, 4) 

Las columnas son:  ['ano', 'mes', 'id_fecha', 'trimestre'] 

+----+---+-----------+---------+
| ano|mes|   id_fecha|trimestre|
+----+---+-----------+---------+
|2012| 10|17179869185|        4|
|2010|  7|42949672961|        3|
|2010| 12|42949672962|        4|
|2015|  2|60129542145|        1|
|2017|  3|85899345921|        1|
+----+---+-----------+---------+
only showing top 5 rows

None


Dimensión aerolinea: 

Las dimensiones de la base de datos son:  (162, 2) 

Las columnas son:  ['empresa', 'id_aerolinea'] 

+--------------------+------------+
|             empresa|id_aerolinea|
+--------------------+------------+
|"SERVICIO AÉREO R...|           1|
|              21 AIR|  8589934593|
|                ABSA| 17179869185|
|ABX AIR INC SUCUR...| 25769803777|
| AER CARIBE LIMITADA| 34359738369|
+--------------------+------------+
only showing top 5 rows

None


Dimensión operacion: 

Las dimensiones de la base de dato